In [1]:
import QUANTAXIS as QA
import sys

if QA.__version__<'1.1.3':
    print('请先升级QUANTAXIS至1.1.3来使用此教程 pip install quantaxis==1.1.3 或者git pull')
    sys.exit()

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.1.4.dev2
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`

In [2]:
Account=QA.QA_Account(allow_sellopen=True,allow_t0=True,account_cookie='future_test',market_type=QA.MARKET_TYPE.FUTURE_CN,frequence=QA.FREQUENCE.FIFTEEN_MIN)

In [3]:
Broker=QA.QA_BacktestBroker()

In [4]:
rb=QA.QA_fetch_get_future_min('tdx','RBL8','2018-01-01','2018-08-28',frequence='15min')

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 115.238.56.198 future_ip 106.14.95.149


In [5]:
rb_ds=QA.QA_DataStruct_Future_min(rb.set_index(['datetime','code']))

In [6]:
import numpy as np
import pandas as pd
def MACD_JCSC(dataframe,SHORT=12,LONG=26,M=9):
    """
    1.DIF向上突破DEA，买入信号参考。
    2.DIF向下跌破DEA，卖出信号参考。
    """
    CLOSE=dataframe.close
    DIFF =QA.EMA(CLOSE,SHORT) - QA.EMA(CLOSE,LONG)
    DEA = QA.EMA(DIFF,M)
    MACD =2*(DIFF-DEA)

    CROSS_JC=QA.CROSS(DIFF,DEA)
    CROSS_SC=QA.CROSS(DEA,DIFF)
    ZERO=0
    return pd.DataFrame({'DIFF':DIFF,'DEA':DEA,'MACD':MACD,'CROSS_JC':CROSS_JC,'CROSS_SC':CROSS_SC,'ZERO':ZERO})

ind=rb_ds.add_func(MACD_JCSC)


In [7]:
_date = None
for items in rb_ds.panel_gen:
    if _date != items.date[0]:
        print('try to settle')
        _date=items.date[0]
        Account.settle()
    
    
    for item in items.security_gen:
        daily_ind=ind.loc[item.index]
        if daily_ind.CROSS_JC.iloc[0]>0:
            order=Account.send_order(
                code=item.code[0], 
                time=item.datetime[0], 
                amount=200, 
                towards=QA.ORDER_DIRECTION.BUY, 
                price=0, 
                order_model=QA.ORDER_MODEL.CLOSE, 
                amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                )

            if order:
                print(order)
                print(item)
                Broker.receive_order(QA.QA_Event(order=order,market_data=item))


                trade_mes=Broker.query_orders(Account.account_cookie,'filled')
                res=trade_mes.loc[order.account_cookie,order.realorder_id]
                order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)
        elif daily_ind.CROSS_SC.iloc[0]>0:
            if Account.sell_available.get(item.code[0], 0)>0:
                order=Account.send_order(
                    code=item.code[0], 
                    time=item.datetime[0], 
                    amount=Account.sell_available.get(item.code[0], 0), 
                    towards=QA.ORDER_DIRECTION.SELL, 
                    price=0, 
                    order_model=QA.ORDER_MODEL.MARKET, 
                    amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                    )
                if order:
                    Broker.receive_order(QA.QA_Event(order=order,market_data=item))


                    trade_mes=Broker.query_orders(Account.account_cookie,'filled')
                    res=trade_mes.loc[order.account_cookie,order.realorder_id]
                    order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)
    Account.settle()
            

try to settle
try to settle
try to settle
< QA_Order realorder_id Order_yNHQpd0D datetime:2018-01-04 09:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_yNHQpd0D account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
receive deal
< QA_Order realorder_id Order_lE3DjOgw datetime:2018-01-04 14:30:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_lE3DjOgw account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
< QA_Order realorder_id Order_C3ItLp06 datetime:2018-01-05 15:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_C3ItLp06 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
< QA_Order realorder_id Order_V9ZFC8EN datetime:2018-01-08 15:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_V9ZFC8EN account:future_

< QA_Order realorder_id Order_iQaDHZEz datetime:2018-03-02 14:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_iQaDHZEz account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
try to settle
< QA_Order realorder_id Order_6QhLTe1y datetime:2018-03-06 09:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_6QhLTe1y account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
try to settle
< QA_Order realorder_id Order_woA7yMjQ datetime:2018-03-08 10:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_woA7yMjQ account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
receive deal
try to settle
< QA_Order realorder_id Order_kJmij0sv datetime:2018-03-09 09:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_kJmij0sv account:future_

< QA_Order realorder_id Order_0soUqvmh datetime:2018-04-25 13:45:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_0soUqvmh account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
try to settle
< QA_Order realorder_id Order_VQTCSz45 datetime:2018-04-27 15:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_VQTCSz45 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
< QA_Order realorder_id Order_7mW5in30 datetime:2018-04-27 21:30:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_7mW5in30 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
try to settle
receive deal
try to settle
< QA_Order realorder_id Order_EljRPNdn datetime:2018-05-04 09:30:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_EljRPNdn account:future_test status:q

< QA_Order realorder_id Order_8cbKvGD0 datetime:2018-06-13 10:45:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_8cbKvGD0 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
receive deal
try to settle
< QA_Order realorder_id Order_AvbhlpeU datetime:2018-06-14 10:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_AvbhlpeU account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
receive deal
< QA_Order realorder_id Order_ObXZGqrF datetime:2018-06-14 14:45:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_ObXZGqrF account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
< QA_Order realorder_id Order_KutpceL7 datetime:2018-06-15 15:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_KutpceL7 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 

< QA_Order realorder_id Order_DPhE0vdr datetime:2018-08-01 23:00:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_DPhE0vdr account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
< QA_Order realorder_id Order_mSD0ny6V datetime:2018-08-02 11:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_mSD0ny6V account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
receive deal
< QA_Order realorder_id Order_BNlo7Zf9 datetime:2018-08-03 11:30:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_BNlo7Zf9 account:future_test status:queued >
< QA_DataStruct_Future_min with 1 securities >
receive deal
try to settle
< QA_Order realorder_id Order_wLrnBip5 datetime:2018-08-06 14:15:00 code:RBL8 amount:200 price:0 towards:1 btype:future_cn order_id:Order_wLrnBip5 account:future_test status:queued >
< QA_DataStruct_Future_min with 1

In [8]:
Account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax
0,2018-01-04 09:15:00,RBL8,3824.0,200,1000000,Order_yNHQpd0D,Order_yNHQpd0D,Trade_2e6Nxp3J,future_test,191.200,0
1,2018-01-04 11:15:00,RBL8,3803.5,-200,1000000,Order_4un3a0zG,Order_4un3a0zG,Trade_IqaA5otJ,future_test,190.175,0
2,2018-01-04 14:30:00,RBL8,3809.0,200,1000000,Order_lE3DjOgw,Order_lE3DjOgw,Trade_w2neS3CE,future_test,190.450,0
3,2018-01-05 09:30:00,RBL8,3793.5,-200,1000000,Order_d15UblHT,Order_d15UblHT,Trade_Fex2sV3D,future_test,189.675,0
4,2018-01-05 15:00:00,RBL8,3798.0,200,1000000,Order_C3ItLp06,Order_C3ItLp06,Trade_iaNqLcU5,future_test,189.900,0
5,2018-01-08 09:30:00,RBL8,3775.0,-200,1000000,Order_mrlRhIDw,Order_mrlRhIDw,Trade_JhgxA1TC,future_test,188.750,0
6,2018-01-08 15:00:00,RBL8,3818.0,200,1000000,Order_V9ZFC8EN,Order_V9ZFC8EN,Trade_lDxgC2PK,future_test,190.900,0
7,2018-01-09 10:45:00,RBL8,3821.0,-200,1000000,Order_LO5xoq8f,Order_LO5xoq8f,Trade_1DCaGvTI,future_test,191.050,0
8,2018-01-10 10:15:00,RBL8,3849.0,200,1000000,Order_FGJ87YAc,Order_FGJ87YAc,Trade_B4mTFN8n,future_test,192.450,0
9,2018-01-11 09:15:00,RBL8,3862.0,-200,1000000,Order_VimRMd1u,Order_VimRMd1u,Trade_pELtsced,future_test,193.100,0


In [9]:
rb_day=QA.QA_fetch_get_future_day('tdx','RBL8','2018-01-01','2018-08-28')
rb_dayds=QA.QA_DataStruct_Future_day(rb_day.set_index(['date','code']))

In [10]:
Risk=QA.QA_Risk(Account,if_fq=False,market_data=rb_dayds)

In [12]:
Risk.plot_assets_curve()

ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000019357DC2AE8> (for post_execute):


ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -0.001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 1008x864 with 2 Axes>

In [13]:
Risk.profit_construct

{'total_buyandsell': nan,
 'total_tax': 0,
 'total_commission': -51191.35,
 'total_profit': nan}